In [378]:
import sklearn
import pandas as pd
import numpy as np

In [379]:
folder = "clickbait_challenge"

In [380]:
dataset = pd.read_csv("dataset/"+folder+"/dataframe/features.csv")

In [381]:
dfx = np.array(dataset)
dfy = np.array(pd.read_csv("dataset/"+folder+"/dataframe/dataset.csv")["Clickbait"])

### MLP Train-test-split

In [382]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.3, random_state=1)

In [383]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation="relu", solver="sgd", hidden_layer_sizes=(7), random_state=1)

In [384]:
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=7, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [385]:
mlp_y_pred = mlp.predict(X_test)

In [386]:
from sklearn import metrics
metrics.accuracy_score(y_test, mlp_y_pred)

0.6482588814632431

### MLP 10-fold CV

In [387]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [388]:
cv_mlp_clf = MLPClassifier(activation="tanh", solver="sgd", hidden_layer_sizes=(7), random_state=1, max_iter=1000)
cvs = cross_val_score(cv_mlp_clf, dfx, dfy, cv=10, scoring='accuracy')

In [389]:
cvs.mean()

0.6363425028626931

### SVM Count Vectorize

In [390]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

In [391]:
titles_for_cv = pd.read_csv("dataset/"+folder+"/dataframe/dataset.csv")["Title"]
count_vec = count_vectorizer.fit_transform(titles_for_cv)
count_vec.shape

(9476, 14886)

In [392]:
from sklearn.model_selection import train_test_split
X_train_SVM, X_test_SVM, y_train_SVM, y_test_SVM = train_test_split(count_vec, dfy, test_size=0.3, random_state=1)

In [393]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')

In [394]:
clf.fit(X_train_SVM, y_train_SVM)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [395]:
svm_y_pred = clf.predict(X_test_SVM)

In [396]:
from sklearn import metrics
metrics.accuracy_score(y_test, svm_y_pred)

0.49876890608512137

### Count Vectorizer with MLP

In [397]:
mlp_count_vec = MLPClassifier(activation="relu", solver="sgd", hidden_layer_sizes=(7), random_state=1)

In [398]:
mlp_count_vec.fit(X_train_SVM, y_train_SVM)

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=7, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [399]:
predict = mlp_count_vec.predict(X_test_SVM)
real_val = y_test_SVM


counter = 0
for i in range(len(predict)):
    if predict[i] == real_val[i]:
        counter += 1
counter/len(predict)

0.7569468870911009

### SVM 10-fold CV

In [400]:
from sklearn import svm

In [401]:
from sklearn.preprocessing import StandardScaler

In [402]:
sc = StandardScaler()  
dfx = sc.fit_transform(dfx)  

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [403]:
cv_svm_clf = svm.SVC(kernel='rbf')
cv_svm_res = cross_val_score(cv_svm_clf, dfx, dfy, cv=10, scoring='accuracy')

In [404]:
cv_svm_res.mean()

0.6551276738891192

### Random Forest 10-fold CV

In [405]:
from sklearn.ensemble import RandomForestClassifier

In [406]:
regressor = RandomForestClassifier(n_estimators=20, random_state=2) 

In [407]:
cv_rf_res = cross_val_score(regressor, dfx, dfy, cv=10, scoring='accuracy')

In [408]:
cv_rf_res.mean()

0.6568165514906055

### Logistic regression

In [409]:
from sklearn.linear_model import LogisticRegression

In [410]:
lr = LogisticRegression(solver='saga', max_iter=1000)

In [411]:
cv_lr_res = cross_val_score(lr, dfx, dfy, cv=10, scoring='accuracy')

In [412]:
cv_lr_res.mean()

0.6493217533494624